In [1]:
import pandas as pd
import numpy as np
import joblib


In [2]:
model = joblib.load("../modelos/lightgbm_model.joblib")
print(" modelo cargado")


 modelo cargado


In [3]:
items = pd.read_csv("../data/archive/items_en.csv")
test = pd.read_csv("../data/archive/test.csv")
monthly = pd.read_csv("../data/new/monthly_dataset.csv")

In [4]:
if "item_category_id" not in monthly.columns:
    monthly = monthly.merge(items[["item_id", "item_category_id"]], on="item_id", how="left")


In [5]:
last_month = monthly["date_block_num"].max()
pred_month = last_month + 1

last_month, pred_month


(np.int64(33), np.int64(34))

In [6]:
pred_base = test[["shop_id", "item_id"]].copy()
pred_base["date_block_num"] = pred_month


In [7]:
pred_base = pred_base.merge(items[["item_id", "item_category_id"]], on="item_id", how="left")
pred_base.head()


,shop_id,item_id,date_block_num,item_category_id
0,5,5037,34,19
1,5,5320,34,55
2,5,5233,34,19
3,5,5232,34,23
4,5,5268,34,20


In [8]:
FEATURES = [
    "lag_1", "lag_2", "lag_3",
    "item_avg_lag_1",
    "shop_month_avg_lag_1",
    "cat_month_avg_lag_1",
    "month_of_year"
]


In [9]:
hist = monthly[["date_block_num", "shop_id", "item_id", "item_cnt_month"]].copy()


In [10]:
lag1 = hist[hist["date_block_num"] == pred_month - 1].rename(columns={"item_cnt_month": "lag_1"})
lag1 = lag1[["shop_id", "item_id", "lag_1"]]
pred_base = pred_base.merge(lag1, on=["shop_id", "item_id"], how="left")


In [11]:
lag2 = hist[hist["date_block_num"] == pred_month - 2].rename(columns={"item_cnt_month": "lag_2"})
lag2 = lag2[["shop_id", "item_id", "lag_2"]]
pred_base = pred_base.merge(lag2, on=["shop_id", "item_id"], how="left")


In [12]:
lag3 = hist[hist["date_block_num"] == pred_month - 3].rename(columns={"item_cnt_month": "lag_3"})
lag3 = lag3[["shop_id", "item_id", "lag_3"]]
pred_base = pred_base.merge(lag3, on=["shop_id", "item_id"], how="left")


In [13]:
for c in ["lag_1", "lag_2", "lag_3"]:
    pred_base[c] = pred_base[c].fillna(0)


In [14]:
item_avg = (
    hist.groupby("item_id")["item_cnt_month"].mean()
    .rename("item_avg_lag_1")
    .reset_index()
)
pred_base = pred_base.merge(item_avg, on="item_id", how="left")
pred_base["item_avg_lag_1"] = pred_base["item_avg_lag_1"].fillna(0)


In [15]:
shop_month_avg_last = (
    hist[hist["date_block_num"] == pred_month - 1]
    .groupby("shop_id")["item_cnt_month"]
    .mean()
    .rename("shop_month_avg_lag_1")
    .reset_index()
)

pred_base = pred_base.merge(shop_month_avg_last, on="shop_id", how="left")
pred_base["shop_month_avg_lag_1"] = pred_base["shop_month_avg_lag_1"].fillna(0)


In [16]:
cat_month_avg_last = (
    monthly[monthly["date_block_num"] == pred_month - 1]
    .groupby("item_category_id")["item_cnt_month"]
    .mean()
    .rename("cat_month_avg_lag_1")
    .reset_index()
)
pred_base = pred_base.merge(cat_month_avg_last, on="item_category_id", how="left")
pred_base["cat_month_avg_lag_1"] = pred_base["cat_month_avg_lag_1"].fillna(0)


In [17]:
pred_base["month_of_year"] = pred_base["date_block_num"] % 12


In [18]:
FEATURES = [
    "lag_1", "lag_2", "lag_3",
    "item_avg_lag_1",
    "shop_month_avg_lag_1",
    "cat_month_avg_lag_1",
    "month_of_year"
]

X_test = pred_base[FEATURES]
pred = model.predict(X_test)
pred = np.clip(pred, 0, 20)


In [19]:
submission = pd.DataFrame({
    "ID": test["ID"],
    "item_cnt_month": pred
})

submission.head()


,ID,item_cnt_month
0,0,1.791042
1,1,0.017993
2,2,1.246507
3,3,0.305178
4,4,0.000000


In [20]:
submission.to_csv("../entregables/submission.csv", index=False)
print("✅ Guardado en ./submissions/submission.csv")


✅ Guardado en ./submissions/submission.csv
